# Processamento de Dados

Este notebook tem como objetivo entender a organização dos objetos JSON ddo arquivo, assim servirá como base para o projeto do banco de dados.

## Importação das bibliotecas

In [1]:
import json
import random
import pandas as pd

## Leitura dos JSONs do arquivo

In [2]:
# Cria uma lista para armazenar os objetos JSON
logsList = []

# Percorre o arquivo lendo cada linha como um objeto json e adicionando à lista
with open('../raw_data/logs.txt') as f:
    for jsonObj in f:
        log = json.loads(jsonObj)
        logsList.append(log)

### Primeira impressão da organização do JSON

Visualização dos dados de um exemplo aletório

In [3]:
r = random.randint(0, len(logsList))

In [4]:
sample = logsList[r]

In [5]:
pd.DataFrame(sample).dtypes

request                  object
upstream_uri             object
response                 object
authenticated_entity     object
route                    object
service                  object
latencies               float64
client_ip                object
started_at                int64
dtype: object

In [6]:
sample

{'request': {'method': 'GET',
  'uri': '/',
  'url': 'http://dibbert.biz',
  'size': 176,
  'querystring': [],
  'headers': {'accept': '*/*',
   'host': 'dibbert.biz',
   'user-agent': 'curl/7.37.1'}},
 'upstream_uri': '/',
 'response': {'status': 200,
  'size': 319,
  'headers': {'Content-Length': '197',
   'via': 'kong/1.3.0',
   'Connection': 'close',
   'access-control-allow-credentials': 'true',
   'Content-Type': 'application/json',
   'server': 'nginx',
   'access-control-allow-origin': '*'}},
 'authenticated_entity': {'consumer_id': {'uuid': 'b235ed88-1a20-3585-a237-a85f288b1067'}},
 'route': {'created_at': 1561154217,
  'hosts': 'schneider.com',
  'id': '7bedb816-c359-3b8c-a0f9-b03b46c63402',
  'methods': ['GET', 'POST', 'PUT', 'DELETE', 'PATCH', 'OPTIONS', 'HEAD'],
  'paths': ['/'],
  'preserve_host': False,
  'protocols': ['http', 'https'],
  'regex_priority': 0,
  'service': {'id': 'd035ffcf-914a-3007-b028-ae18f04d75b4'},
  'strip_path': True,
  'updated_at': 1561154217},
 

## Varredura das chaves dos JSONs

Assumindo que as chaves no primeiro nível são constantes e presentes em todos exemplos, vamos verificar apenas as subchaves.

In [22]:
# chaves do primeiro nível
main_keys = sample.keys()

subkeys = {}

for k in main_keys:
    subkeys[k] = set()

In [23]:
for jsonObj in logsList:
    for k in main_keys:
        if type(jsonObj[k]) == dict:
            subkeys[k].update(jsonObj[k].keys())

Tendo varrido o conjunto das subchaves, vamos utilizá-los como referência para criação do modelo relacional.

In [26]:
for k,v in subkeys.items():
    print("{}: {}".format(k, v))
    print()

request: {'method', 'size', 'headers', 'uri', 'url', 'querystring'}

upstream_uri: set()

response: {'size', 'status', 'headers'}

authenticated_entity: {'consumer_id'}

route: {'methods', 'regex_priority', 'preserve_host', 'hosts', 'strip_path', 'protocols', 'paths', 'updated_at', 'created_at', 'id', 'service'}

service: {'path', 'protocol', 'retries', 'port', 'connect_timeout', 'read_timeout', 'host', 'name', 'updated_at', 'created_at', 'id', 'write_timeout'}

latencies: {'proxy', 'kong', 'request'}

client_ip: set()

started_at: set()

